In [ ]:
# Obtain an AAD token with the correct resource ID
import json
import requests

#Setup the endpoint
endpoint = 'https://login.microsoftonline.com/[TENANT_ID]/oauth2/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
payload = 'grant_type=client_credentials&client_id=[SERVICE_PRINCIPAL_CLIENT_ID]&client_secret=[SERVICE_PRINCIPAL_SECRET]&resource=2ff814a6-3304-4ab8-85cb-cd0e6f879c1d'
r = requests.post(endpoint, headers=headers, data=payload)
response = r.json()
#print(response)
bearertoken = response["access_token"]
print(bearertoken)

In [ ]:
# Create a databricks access token
endpoint = 'https://[DATABRICKS_WORKSPACE_URL]/api/2.0/token/create'
payload={
  "lifetime_seconds": 300,
  "comment": "ADF Token"
}
headers = {'X-Databricks-Org-Id':'[DATABRICKS_ORG_ID]','Authorization': 'Bearer %s' % bearertoken}
r = requests.post(endpoint, headers=headers, data=json.dumps(payload))
response = r.json()
adb_token = response["token_value"]

In [ ]:
# Optional request to list the tokens. These tokens created via service principal are not diplayed in workspace UI as the personal access token is generated by a user account
endpoint = 'https://[DATABRICKS_WORKSPACE_URL]/api/2.0/token/list'
headers = {'X-Databricks-Org-Id':'[DATABRICKS_ORG_ID]','Authorization': 'Bearer %s' % bearertoken}
r = requests.get(endpoint, headers=headers)
response = r.json()
print(response)

In [ ]:
# Obtain an auth token for keyvault
# to store the Databricks access token
endpoint = 'https://login.microsoftonline.com/[TENANT_ID]/oauth2/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
payload = 'grant_type=client_credentials&client_id=[SERVICE_PRINCIPAL_CLIENT_ID]&client_secret=[SERVICE_PRINCIPAL_SECRET]&resource=https%3A%2F%2Fvault.azure.net'
r = requests.post(endpoint, headers=headers, data=payload)
response = r.json()
bearertoken = response["access_token"]
print(bearertoken)

In [ ]:
endpoint = 'https://[KEY VAULT].vault.azure.net/secrets/[SECRET NAME]?api-version=7.0'
payload={
"value": adb_token
}
headers = {'Content-Type': 'application/json','Authorization': 'Bearer %s' % bearertoken}
r = requests.put(endpoint, headers=headers,data=json.dumps(payload))
print(r.text)